# Cryptomaven herontdekken

Wat deed ik ook alweer waardoor het zo goed werkte..?

In [1]:
import pandas as pd
import pandas_ta as ta
import numpy as np


In [2]:
# Load data
df = pd.read_json("ETH_USDT-1d.json")
df.columns=['date','open','high','low','close','volume']
df['date']=(pd.to_datetime(df['date'],unit='ms'))
df.tail(5)

,date,open,high,low,close,volume
994,2021-05-28,2742.23,2762.45,2327.38,2412.10,1.554585e+06
995,2021-05-29,2412.34,2574.27,2204.10,2278.74,1.379427e+06
996,2021-05-30,2278.74,2476.42,2180.53,2385.98,1.151113e+06
997,2021-05-31,2385.82,2720.00,2271.44,2706.15,1.307958e+06
998,2021-06-01,2706.15,2740.00,2523.74,2634.57,1.179577e+06


In [5]:
# Default Pandas-ta indicators
df.ta.rsi(length=10, append=True)
df.ta.mfi(length=10, append=True)
df.tail(5)

,date,open,high,low,close,volume,RSI_10,MFI_10
994,2021-05-28,2742.23,2762.45,2327.38,2412.10,1.554585e+06,39.868661,39.945526
995,2021-05-29,2412.34,2574.27,2204.10,2278.74,1.379427e+06,37.649240,46.268607
996,2021-05-30,2278.74,2476.42,2180.53,2385.98,1.151113e+06,40.603558,34.206584
997,2021-05-31,2385.82,2720.00,2271.44,2706.15,1.307958e+06,48.671381,45.121599
998,2021-06-01,2706.15,2740.00,2523.74,2634.57,1.179577e+06,47.082738,53.613276


In [6]:
# Hier heb ik functies gemaakt die controleren of een waarde overschreden wordt:

# RSI check
def x(row):
    if row["RSI_10"] < 32:
        val = 1
    elif row["RSI_10"] > 75:
        val = 3
    else:
        val = 0
    return val
df["RSI_pos"] = df.apply(x, axis=1)
# MFI check
def y(row):
    if row["MFI_10"] < 21:
        val = 1
    elif row["MFI_10"] > 85:
        val = 3
    else:
        val = 0
    return val
df["MFI_pos"] = df.apply(y, axis=1)

In [10]:
df[(df['RSI_pos'] > 0) | (df['MFI_pos'] > 0)].tail(5)

,date,open,high,low,close,volume,RSI_10,MFI_10,RSI_pos,MFI_pos
975,2021-05-09,3910.26,3983.65,3726.08,3923.95,1.000401e+06,83.192669,92.232364,3,3
976,2021-05-10,3923.95,4208.00,3650.00,3945.88,1.505898e+06,83.482908,93.050366,3,3
977,2021-05-11,3946.63,4180.22,3771.64,4173.45,1.095927e+06,86.225532,93.469733,3,3
985,2021-05-19,3375.08,3441.73,1888.00,2438.92,4.309836e+06,30.707759,32.384655,1,0
989,2021-05-23,2295.45,2381.57,1728.74,2096.39,2.914549e+06,30.080250,28.036315,1,0


In [12]:
# Posities tellen om te berekenen waar ik kan instappen
df["POS_cnt"] = df["RSI_pos"] + df["MFI_pos"]
df[["RSI_pos", "MFI_pos", "POS_cnt"]].tail(15)

,RSI_pos,MFI_pos,POS_cnt
984,0,0,0
985,1,0,1
986,0,0,0
987,0,0,0
988,0,0,0
989,1,0,1
990,0,0,0
991,0,0,0
992,0,0,0
993,0,0,0


In [13]:
# Advies maken op basis van berekeningen
# Determine advice
def f(row):
    if row["POS_cnt"] == 0:
        val = "NEUTRAL"
    elif row["POS_cnt"] == 2:
        val = "BUY"
    elif row["POS_cnt"] == 6:
        val = "SELL"
    else:
        val = "NEUTRAL"
    return val
# create new column 'POS-adv' using the function above
df["POS_adv"] = df.apply(f, axis=1)

df[(df['RSI_pos'] > 0) | (df['MFI_pos'] > 0)].tail(5)

,date,open,high,low,close,volume,RSI_10,MFI_10,RSI_pos,MFI_pos,POS_cnt,POS_adv
975,2021-05-09,3910.26,3983.65,3726.08,3923.95,1.000401e+06,83.192669,92.232364,3,3,6,SELL
976,2021-05-10,3923.95,4208.00,3650.00,3945.88,1.505898e+06,83.482908,93.050366,3,3,6,SELL
977,2021-05-11,3946.63,4180.22,3771.64,4173.45,1.095927e+06,86.225532,93.469733,3,3,6,SELL
985,2021-05-19,3375.08,3441.73,1888.00,2438.92,4.309836e+06,30.707759,32.384655,1,0,1,NEUTRAL
989,2021-05-23,2295.45,2381.57,1728.74,2096.39,2.914549e+06,30.080250,28.036315,1,0,1,NEUTRAL


In [16]:
# Misschien wel de belangrijkste kolom van allemaal.
# Hier wordt namelijk het signaal gegenereerd waarop gereageerd dient te worden.


# Create columns that indicates that a position advice has changed
# This disables the need for comparison with a 'last_advice' file
df["POS_adv_changed"] = df["POS_adv"].shift(+1) == df["POS_adv"]

df.tail(50)

,date,open,high,low,close,volume,RSI_10,MFI_10,RSI_pos,MFI_pos,POS_cnt,POS_adv,POS_adv_changed
949,2021-04-13,2137.86,2318.00,2135.53,2299.19,7.826086e+05,72.125297,77.825192,0,0,0,NEUTRAL,True
950,2021-04-14,2299.19,2447.29,2281.33,2432.60,9.729316e+05,77.067800,87.288699,3,3,6,SELL,False
951,2021-04-15,2432.59,2543.47,2400.00,2514.04,6.819510e+05,79.529687,87.527952,3,3,6,SELL,True
952,2021-04-16,2514.04,2548.29,2300.00,2422.67,9.991638e+05,70.142663,72.597477,0,0,0,NEUTRAL,False
953,2021-04-17,2422.38,2495.00,2310.00,2317.60,6.242323e+05,60.950668,74.093943,0,0,0,NEUTRAL,True
954,2021-04-18,2317.65,2340.00,1946.80,2235.64,1.475384e+06,54.733901,58.786072,0,0,0,NEUTRAL,True
955,2021-04-19,2235.65,2300.00,2080.00,2161.12,8.205923e+05,49.620865,60.842019,0,0,0,NEUTRAL,True
956,2021-04-20,2161.12,2346.62,2055.00,2330.03,9.922408e+05,59.216021,62.553480,0,0,0,NEUTRAL,True
957,2021-04-21,2330.26,2469.54,2235.16,2357.06,1.177886e+06,60.551926,66.011151,0,0,0,NEUTRAL,True
958,2021-04-22,2357.06,2644.47,2303.44,2397.94,2.220753e+06,62.610011,71.686681,0,0,0,NEUTRAL,True


In [25]:
# # Onderstaande functie is iets aangepast tav de CryptoMaven functie
# # Nu zal bij elk signal een True gegeven worden in de nieuwe kolom 'signal'
# df['signal'] = False
# def check_swing_strategy(df):
#     # Get the second to last entry in the dataframe column with advice
#     # This is the last full closing day.
#     change = df.iloc[-2]["POS_adv_changed"]
    
#     # If change field is false (not the same as the previous advice),
#     # then the advice has been changed and a warning should be created.
#     if change == False:
#         # return swing strategy advice
#         df['signal'] = True
    
#     return df

# check_swing_strategy(df)
change = df.iloc[-2]["POS_adv_changed"]
if change == False:
    df['signal'] = True

print(change)
df.tail(5)

True


,date,open,high,low,close,volume,RSI_10,MFI_10,RSI_pos,MFI_pos,POS_cnt,POS_adv,POS_adv_changed,signal
994,2021-05-28,2742.23,2762.45,2327.38,2412.10,1.554585e+06,39.868661,39.945526,0,0,0,NEUTRAL,True,False
995,2021-05-29,2412.34,2574.27,2204.10,2278.74,1.379427e+06,37.649240,46.268607,0,0,0,NEUTRAL,True,False
996,2021-05-30,2278.74,2476.42,2180.53,2385.98,1.151113e+06,40.603558,34.206584,0,0,0,NEUTRAL,True,False
997,2021-05-31,2385.82,2720.00,2271.44,2706.15,1.307958e+06,48.671381,45.121599,0,0,0,NEUTRAL,True,False
998,2021-06-01,2706.15,2740.00,2523.74,2634.57,1.179577e+06,47.082738,53.613276,0,0,0,NEUTRAL,True,False
